In [109]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [110]:
df_original = pd.read_csv("dadosProb.csv", encoding="ISO-8859-1") #Fazendo os dados do excel serem lidos

In [111]:
print(df_original.columns)

Index(['id', 'data', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio',
       'CAUSA_ACIDENTE', 'TIPO_ACIDENTE', 'classificacao_acidente', 'fase_dia',
       'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via',
       'uso_solo', 'pessoas', 'mortos', 'feridos_leves', 'feridos_graves',
       'ilesos', 'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude',
       'regional', 'delegacia', 'uop'],
      dtype='object')


In [121]:
def filtrar_por_br(df, nome_br):
    """
    Filtra o DataFrame para incluir apenas os registros de uma BR específica.
    
    Args:
        df (pd.DataFrame): O DataFrame original.
        nome_br (str): O valor da BR a ser filtrado (ex: '101' ou '230').
        
    Returns:
        pd.DataFrame: O DataFrame filtrado.
    """
    # Garante que a coluna 'br' existe antes de filtrar
    if 'br' not in df.columns:
        print("Erro: A coluna 'br' não foi encontrada no DataFrame.")
        return pd.DataFrame() # Retorna um DataFrame vazio em caso de erro

    # Cria uma máscara booleana e usa-a para selecionar apenas as linhas desejadas
    df_filtrado = df[df['br'] == nome_br].copy()
    
    return df_filtrado
df_filtrado = filtrar_por_br(df_original,230)

In [128]:
inicio = pd.to_datetime(df_filtrado['data'],format="%d/%m/%Y").dt.date.min() #periodo de amostragem
fim = pd.to_datetime(df_filtrado['data'], format="%d/%m/%Y").dt.date.max()
print('Período dos dados - De:', inicio, 'Até:',fim)

Período dos dados - De: 2024-01-01 Até: 2024-12-31


In [114]:
df_original['pessoas'].mean() # soma todas as pessoas e divide pela respectivas quantidade

np.float64(2.330208333333333)

In [115]:
def analisar_distribuicao(df_original, tipo_pista):
    """
    Recebe um dataframe e o nome da coluna.
    Retorna uma tabela com as categorias, quantidade e porcentagem.
    """
    # Verifica se a coluna existe para evitar erros
    if tipo_pista not in df_original.columns:
        return f"Erro: A coluna '{tipo_pista}' não foi encontrada."

    # Calcula a contagem absoluta (quantas vezes aparece)
    contagem = df_original[tipo_pista].value_counts()
    
    # Calcula a porcentagem (frequência relativa)
    porcentagem = df_original[tipo_pista].value_counts(normalize=True) * 100
    
    # Cria um DataFrame novo para exibir como tabela bonita
    tabela = pd.Dataframe({
        'Categoria': contagem.index,
        'Quantidade': contagem.values,
        'Porcentagem (%)': porcentagem.values
    })
    
    # Arredonda a porcentagem para 2 casas decimais
    tabela['Porcentagem (%)'] = tabela['Porcentagem (%)'].round(2)
    
    return tabela



In [129]:

# --- A FUNÇÃO DE ANÁLISE ---
def analisar_distribuicao(dt_original, tipo_pista):
    """
    Recebe um dt_original e o nome da tipo_pista.
    Retorna uma tabela (DataFrame) com as categorias, quantidade e porcentagem.
    """
    if tipo_pista not in dt_original.columns:
        # Se a tipo_pista não existir, retorna a mensagem de erro.
        return f"Erro: A tipo_pista '{tipo_pista}' não foi encontrada."

    # 1. Calcula a contagem absoluta
    contagem = dt_original[tipo_pista].value_counts()
    
    # 2. Calcula a porcentagem
    porcentagem = dt_original[tipo_pista].value_counts(normalize=True) * 100
    
    # 3. Cria o DataFrame de resultado
    tabela = pd.DataFrame({
        'Categoria': contagem.index,
        'Quantidade': contagem.values,
        'Porcentagem (%)': porcentagem.values
    })
    
    # 4. Arredonda e retorna
    tabela['Porcentagem (%)'] = tabela['Porcentagem (%)'].round(2)
    return tabela

# ----------------------------------------------------
# --- EXECUÇÃO E DIAGNÓSTICO (Onde o print está) ---
# ----------------------------------------------------

try:
    # Tenta carregar o arquivo. Se falhar, vai para o bloco 'except'.
    df_original = pd.read_csv("dadosProb.csv", encoding="ISO-8859-1")
    
    print("✅ Arquivo CSV carregado com sucesso.")
    
    # --- CHAMADA E PRINT DO RESULTADO (Onde o print DEVE estar) ---
    tipo_pista_a_analisar = 'tipo_pista' # Nome da tipo_pista que queremos analisar
    
    # 1. Chama a função e armazena o resultado retornado
    resultado_pista = analisar_distribuicao(df_original, tipo_pista_a_analisar)
    
    # 2. Imprime o título e o resultado
    print(f"\n--- Análise de Distribuição de {tipo_pista_a_analisar} ---")
    print(resultado_pista) # ESTA LINHA IMPRIME A TABELA RETORNADA PELA FUNÇÃO

except FileNotFoundError:
    print("\n❌ ERRO FATAL: Arquivo não encontrado. O 'dadosProb.csv' está no mesmo diretório?")
except UnicodeDecodeError:
    print("\n❌ ERRO FATAL: Problema de codificação. Tente mudar o 'encoding' para 'utf-8' ou 'latin-1'.")
except Exception as e:
    print(f"\n❌ ERRO FATAL INESPERADO: Ocorreu um erro: {e}")

✅ Arquivo CSV carregado com sucesso.

--- Análise de Distribuição de tipo_pista ---
   Categoria  Quantidade  Porcentagem (%)
0      Dupla        1162            60.52
1    Simples         699            36.41
2  MÃºltipla          59             3.07
